In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from keras.preprocessing.sequence import TimeseriesGenerator

In [6]:
import snowflake.connector as sf
#Define the S3 bucket and Snowflake parameters
snowflake_account = 'at67872.ap-south-1.aws'
snowflake_user = 'FYP4'
snowflake_password = '!Q2w3e4r5t6y'
snowflake_database = 'COAPDATA'
snowflake_schema = 'PUBLIC'
snowflake_warehouse = 'COMPUTE_WH'
snowflake_table = 'monthly_consumption'

In [7]:
# Define the model parameters
n_input = 12
n_features = 1
model_file_name = 'monthly_model.h5'
#load the initial LSTM model
model = load_model(model_file_name)

In [8]:
#Connect to Snowflake and retrieve the latest data
#Connect to Snowflake and retrieve the latest data
conn = sf.connect(user=snowflake_user, password=snowflake_password, account=snowflake_account)
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {snowflake_database}.{snowflake_schema}.{snowflake_table} ORDER BY date DESC")
data = cursor.fetchall()
cursor.close()
conn.close()
df = pd.DataFrame(data, columns=['Month', 'consumption'])
df.set_index('Month', inplace=True)
df.sort_index(inplace=True)

In [9]:
df

,consumption
Month,
2022-01-01,1250.0
2022-02-01,1500.0
2022-03-01,1700.0
2022-04-01,1800.0
2022-05-01,1900.0
2022-06-01,2000.0
2022-07-01,950.0
2022-08-01,1200.0
2022-09-01,1250.0


In [22]:
print(df.dtypes)

consumption    float64
dtype: object


In [10]:
# Scale the data using the same scaler used for training
scaler.fit(df)
scaled_data = scaler.transform(df)

In [7]:
scaled_data

array([[0.81069959],
       [0.62139918],
       [0.58847737],
       [1.        ],
       [0.95884774],
       [0.88888889],
       [0.59259259],
       [0.72427984],
       [0.66255144],
       [0.6090535 ],
       [0.17695473],
       [0.        ]])

In [11]:
# Update the model with the new data
generator = TimeseriesGenerator(scaled_data, scaled_data, length=n_input, batch_size=1)

In [12]:
# take the last 10 and reshape
last_scaled_batch = scaled_data[-12:]
last_scaled_batch = last_scaled_batch.reshape((1, n_input, n_features))

In [13]:
model.fit(generator, epochs=1, verbose=0)
model.save('monthly_model.h5')
    


In [14]:
prediction = model.predict(last_scaled_batch)
true_prediction = scaler.inverse_transform(prediction)[0][0]

In [30]:
true_prediction

859.0288

In [17]:
snowflake_conn = sf.connect(
            user='FYP4',
            password='!Q2w3e4r5t6y',
            account='at67872.ap-south-1.aws',
            warehouse='COMPUTE_WH',
            database='COAPDATA',
            schema='PUBLIC'
    )


cursor = snowflake_conn.cursor()
#cursor.execute(f"INSERT INTO monthly_predict_table (date, prediction) VALUES (DATEADD(DAY, 1, (SELECT MAX(date) FROM monthly_consumption)), {true_prediction})")
cursor.execute(f"INSERT INTO monthly_predict_table (date, prediction) VALUES (DATEADD(MONTH, 1, (SELECT MAX(date) FROM monthly_consumption)), {true_prediction})")
cursor.close()

#cursor.close()
snowflake_conn.close()
    